In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/si699-music-tagging/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/si699-music-tagging


In [ ]:
# %%python3 preprocessing/convert_npy.py

In [ ]:
import os
import random
import platform
import torch
import librosa
from tqdm import tqdm
import numpy as np
import glob
from sklearn.preprocessing import LabelBinarizer
import torchaudio
import csv
import collections
import time
random.seed(42)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, tag_file, npy_root, config, tags, data_type, feature_extractor_type):
        # assert len(filenames) == len(labels), f'Inconsistent length of filenames and labels.'
        self.npy_root = npy_root
        self.config = config
        self.tag_file = tag_file
        self.tags = tags
        self.mlb = LabelBinarizer().fit(self.tags)
        self.data = []
        self.labels = []
        self.data_type = data_type
        # transform waveform into spectrogram
        self.prepare_data()
        self.feature_extractor_type = feature_extractor_type
        # print('Dataset will yield mel spectrogram {} data samples in shape (1, {}, {})'.format(len(self.data),
        #                                                                                        self.config['n_mels'],
        #                                                                                        self.length))
        # self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        assert 0 <= index < len(self)
        waveform = self.data[index]
        target = self.labels[index]
        if self.feature_extractor_type == 'melspec':
            mel_spec = librosa.feature.melspectrogram(y=waveform, sr=self.config['sample_rate'])
            mel_spec = torch.Tensor(mel_spec)
        if self.feature_extractor_type == 'autoextractor':
            feature_extractor = AutoFeatureExtractor.from_pretrained(
                "MIT/ast-finetuned-audioset-10-10-0.4593",
                sampling_rate=self.config['sample_rate'],
                num_mel_bins=self.config['n_mels']
            )
            encoding = feature_extractor(waveform, sampling_rate=self.config['sample_rate'], annotations=target, return_tensors="pt")
            mel_spec = encoding['input_values']
        return mel_spec, target

    def read_file(self):
        tracks = {}
        with open(self.tag_file) as fp:
            reader = csv.reader(fp, delimiter='\t')
            next(reader, None)  # skip header
            for row in reader:
                if not os.path.exists(os.path.join(self.npy_root, row[3].replace('.mp3', '.npy'))):
                    continue
                track_id = row[3].replace('.mp3', '.npy')
                tags = []
                for tag in row[5:]:
                    tags.append(tag.split('---')[-1])
                tracks[track_id] = tags
        return tracks

    def prepare_data(self):
        tracks_dict = self.read_file()
        whole_filenames = sorted(glob.glob(os.path.join(self.npy_root, "*/*.npy")))
        train_size = int(len(whole_filenames) * 0.2)
        val_size = int(len(whole_filenames) * 0.95)
        filenames = []
        random.shuffle(whole_filenames)
        if self.data_type == 'train':
            filenames = whole_filenames[:train_size]
        if self.data_type == 'valid':
            filenames = whole_filenames[val_size:]
        for filename in tqdm(filenames):
            waveform = np.load(filename)
            self.data.append(waveform)
            id = os.path.join(filename.split('/')[-2], filename.split('/')[-1])
            self.labels.append(np.sum(self.mlb.transform(tracks_dict[id]), axis=0))



In [ ]:
from run.models import *
from sklearn.metrics import *
import numpy as np
import collections
import torch
import yaml
from tqdm import tqdm
import logging
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Run on:", device)

Run on: cuda


In [ ]:
def train(model, epoch, criterion, optimizer, train_loader):
    losses = []
    ground_truth = []
    prediction = []
    model.train()
    for input, label in tqdm(train_loader):
        input, label = input.to(device), label.to(device)
        # input = input.unsqueeze(1)
        output = model(input)
        output = output.to(device)
        # print("label:", label.shape, "output", output.shape)
        loss = criterion(output, label.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().detach())
        ground_truth.append(label)
        prediction.append(output)
    get_eval_metrics(prediction, ground_truth, 'train', epoch, losses)
    return model


@torch.no_grad()
def validate(model, epoch, learning_rate, criterion, val_loader, best_pre):
    losses = []
    ground_truth = []
    prediction = []
    model.eval()
    for input, label in tqdm(val_loader):
        input, label = input.to(device), label.to(device)
        # input = input.unsqueeze(1)
        output = model(input)
        output = output.to(device)
        loss = criterion(output, label.float())
        losses.append(loss.cpu().detach())
        ground_truth.append(label)
        prediction.append(output)
    pre = get_eval_metrics(prediction, ground_truth, 'val', epoch, losses)
    if pre > best_pre:
        print("Best precision:", pre)
        best_pre = pre
        torch.save(model, 'model/{}_best_score_{}.pt'.format(model_name, learning_rate))
    return best_pre


def get_eval_metrics(outputs, labels, run_type, epoch, losses):
    outputs = torch.cat(outputs, dim=0)
    labels = torch.cat(labels, dim=0)
    assert outputs.shape == labels.shape
    # 1. number of correctly predicted tags divided by the total number of tags
    prob_classes = []
    for i in range(labels.size(0)):
        label = labels[i]
        k = label.sum()
        _, idx = outputs[i].topk(k=k)
        predict = torch.zeros_like(outputs[i])
        predict[idx] = 1
        prob_classes.append(predict)
    prob_classes = torch.stack(prob_classes)
    matched_1s = torch.mul(prob_classes, labels)
    correct_tag_percentage = matched_1s.sum() / labels.sum()

    # 2. accuracy if set to 1 if exceeds threshold
    # threshold_classes = outputs
    # threshold_classes[threshold_classes > args.threshold] = 1
    # threshold_classes[threshold_classes <= args.threshold] = 0
    # acc = (threshold_classes == labels).sum() / len(threshold_classes.reshape(-1))

    # 3. avg precision
    # avg_pre = average_precision_score(labels.cpu().detach().numpy(), outputs.cpu().detach().numpy(), average='macro')

    # write tensorboard and logging file
    writer.add_scalar("Loss/{}".format(run_type), np.mean(losses), epoch)
    # writer.add_scalar("Pre/{}".format(run_type), avg_pre, epoch)
    writer.add_scalar("Avg_percent/{}".format(run_type), correct_tag_percentage, epoch)
    print("{} - epoch: {}, loss: {}, avg percent: {}".format(
        run_type, epoch, np.mean(losses), correct_tag_percentage))
    logging.info("{} - epoch: {}, loss: {}, avg percent: {}".format(
        run_type, epoch, np.mean(losses), correct_tag_percentage))
    return correct_tag_percentage


def get_model(model_name, tags):
    n_classes = len(tags)
    if model_name =='samplecnn':
        model = SampleCNN(n_classes, config).to(device)
    elif model_name == 'crnn':
        model = CRNN(n_classes, config).to(device)
    elif model_name =='fcn':
        model = FCN(n_classes, config).to(device)
    elif model_name == 'musicnn':
        model = Musicnn(n_classes, config).to(device)
    elif model_name == 'shortchunkcnn_res':
        model = ShortChunkCNN_Res(n_classes, config).to(device)
    elif model_name == 'cnnsa':
        model = CNNSA(n_classes, config).to(device)
    else:
        model = SampleCNN(n_classes, config).to(device)
    return model

def get_tags(tag_file):
    collected_tags = []
    with open(tag_file, 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            result = row[0].split('\t')
            for splitted in result:
                if 'mood/theme---' in splitted:
                    collected_tags.append(splitted.split('---')[-1])
    collected_tags = collections.Counter(collected_tags)
    print(collected_tags.keys())
    return list(collected_tags.keys())

In [ ]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
print("Preparing dataset...")
tag_file = 'data/autotagging_moodtheme.tsv'
npy_root = 'data/npy'
batch_size = 4
tags = get_tags(tag_file)
train_dataset = MyDataset(tag_file, npy_root, config, tags, "train")
val_dataset = MyDataset(tag_file, npy_root, config, tags, "valid")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

Preparing dataset...
dict_keys(['background', 'film', 'melancholic', 'calm', 'melodic', 'children', 'relaxing', 'meditative', 'cool', 'documentary', 'emotional', 'space', 'love', 'drama', 'adventure', 'heavy', 'dark', 'soft', 'energetic', 'retro', 'ballad', 'advertising', 'epic', 'action', 'dramatic', 'powerful', 'upbeat', 'inspiring', 'uplifting', 'soundscape', 'slow', 'deep', 'fun', 'horror', 'nature', 'funny', 'happy', 'positive', 'sad', 'summer', 'dream', 'romantic', 'hopeful', 'mellow', 'motivational', 'party', 'groovy', 'corporate', 'sport', 'travel', 'sexy', 'movie', 'fast', 'commercial', 'holiday', 'ambiental', 'christmas', 'game', 'trailer'])


100%|██████████| 487/487 [11:50<00:00,  1.46s/it]


Dataset will yield mel spectrogram 487 data samples in shape (1, 128, 10240)


100%|██████████| 122/122 [02:46<00:00,  1.36s/it]

Dataset will yield mel spectrogram 122 data samples in shape (1, 128, 10240)


In [ ]:
for input, label in train_loader:
    print(label)
    break

tensor([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
model_name = 'musiccnn'
learning_rate = 1e-4
num_epochs = 50
model = get_model(model_name, tags)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print("Training and validating model...")
writer = SummaryWriter('runs/{}_{}_{}'.format(model_name, learning_rate, batch_size))
logging.basicConfig(filename="log_{}_{}_{}".format(model_name, learning_rate, batch_size),
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    filemode='w',
                    level=logging.INFO)
best_pre = float('-inf')
for epoch in range(num_epochs):
    train(model, epoch, criterion, optimizer, train_loader)
    best_pre = validate(model, epoch, learning_rate, criterion, val_loader, best_pre)
writer.close()
# torch.save(model, 'model/{}.pt'.format(model_name))

Training and validating model...


100%|██████████| 122/122 [00:01<00:00, 68.71it/s]


train - epoch: 0, loss: 0.24712103605270386, avg percent: 0.06408839672803879


100%|██████████| 31/31 [00:00<00:00, 110.13it/s]


val - epoch: 0, loss: 0.14116379618644714, avg percent: 0.13656388223171234
Best precision: tensor(0.1366, device='cuda:0')


...

val - epoch: 48, loss: 0.18674038350582123, avg percent: 0.10572687536478043


100%|██████████| 122/122 [00:01<00:00, 70.18it/s]


train - epoch: 49, loss: 0.007031000219285488, avg percent: 0.9911602139472961


100%|██████████| 31/31 [00:00<00:00, 115.40it/s]

val - epoch: 49, loss: 0.19586223363876343, avg percent: 0.06607929617166519
